## Embedding model ##
>Floris Menninga \
>Datum: 8-01-2026

In [5]:
# %pip install tensorflow keras
# #%pip install tqdm
# %pip install tensorflow_text
# %pip install bs4
# %pip install joblib
# %pip install lxml

from data_parser import xml_parser
import EmbeddingModel

import io
import re
import string
import tqdm
import os

import numpy as np
import requests
import tensorflow as tf
import tensorflow_text as tf_text
import joblib
import tensorflow as tf
from tensorflow.keras import layers
import lxml

%load_ext tensorboard
AUTOTUNE = tf.data.AUTOTUNE
SEED = 0
BATCH_SIZE = 256
BUFFER_SIZE = 10000

2026-01-29 20:20:58.171207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769714458.225788    4971 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769714458.241677    4971 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769714458.350537    4971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769714458.350557    4971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769714458.350560    4971 computation_placer.cc:177] computation placer alr

In [6]:
os.environ['LD_LIBRARY_PATH'] = '/run/opengl-driver/lib:' + os.environ.get('LD_LIBRARY_PATH', '')
print("Gpu's: ", len(tf.config.list_physical_devices('GPU')))

Gpu's:  0


2026-01-29 20:21:00.696013: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2026-01-29 20:21:00.696162: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2026-01-29 20:21:00.696173: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: FLORIS-2
2026-01-29 20:21:00.696182: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] hostname: FLORIS-2
2026-01-29 20:21:00.696237: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:197] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program. The library may be missing or provided via another object.
2026-01-29 20:21:00.696392: I exte

## Tokenizer

De eerste stap is het omzetten van de text in tokens:
In tegenstelling to methodes zoals bytepair encoding etc. worden de tokens gemaakt door de zinnen \
te splitten op spaties.

In [7]:
sentence = "Hallo, dit is een testzin om het programma te testen."
tokens = list(sentence.lower().split())
print(len(tokens))

10


### Vocabulary maken van deze tokens:

In [8]:
vocab, index = {}, 1 
vocab['<pad>'] = 0
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)

print(vocab)

{'<pad>': 0, 'hallo,': 1, 'dit': 2, 'is': 3, 'een': 4, 'testzin': 5, 'om': 6, 'het': 7, 'programma': 8, 'te': 9, 'testen.': 10}


### Inverse vocabulary:
Van integer index naar token:

In [9]:
inverse_vocab = {index: token for token, index in vocab.items()}

print(inverse_vocab)

{0: '<pad>', 1: 'hallo,', 2: 'dit', 3: 'is', 4: 'een', 5: 'testzin', 6: 'om', 7: 'het', 8: 'programma', 9: 'te', 10: 'testen.'}


## Vectorizeren van de zin:

In [10]:
test_sequence = [vocab[word] for word in tokens]

print(test_sequence)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Skip-gram maken:
Het embedding model zal gemaakt worden met behulp van het Word2Vec algorithme. Verdere uitleg over hoe dit algorithme werkt komt straks maar om dit te kunnen gebruiken 
moet er eerst een Skip-gram model gemaatk worden. 
Het belangrijkste punt van een skip-gram is dat de vector representatie gebaseerd is op de ruimtelijke nabijheid van woorden in een zin. 
Als het woord "Ketting" altijd gevolgd zou worden door het woord "zaag" zal deze combinatie van woorden vaker voorkomen dan in andere combinaties.
Een skip-gram model is een klein neural network met een inputlayer, embedding layer en output layer. 
Dit model moet een waarschijnlijkheids verdelings vector geven voor een gegeven input woord. Dat is dus de kans dat deze twee woorden samen voorkomen binnen de context lengte waarmee het model getrained is. De som van deze waarschijnlijkheids verdeling is 1.

In [11]:
window_size = 2

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      test_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0,
      seed=0
)

print(len(positive_skip_grams))

34


Hieronder staan vijf van de gegenereerde skip-grams. 
Zoals in de bovenstaande code gedefinieerde "window size" bestaat elk skip-gram uit twee tokens. 
In dit geval zijn het hele woorden omdat het zo getokenizeerd is. 
Later wil ik nog proberen om andere tokenizatie technieken te gebruiken zoals byte pair encoding of wordpiece / sentencepiece.


In [12]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")


(3, 4): (is, een)
(1, 2): (hallo,, dit)
(8, 9): (programma, te)
(6, 8): (om, programma)
(4, 3): (een, is)


### Negative sampling

De spikgrams functie retourneerd alle positieve skip-gram paren door met een sliding window van 
een gegeven grootte over de tekst te gaan. Voor training is dit echter niet genoeg, er moeten ook negatieve samples bij zitten. Deze negatieve skip-gram paren worden verkregen door willekeurige woorden uit de vocabulary te halen en deze samen te voegen. 

Nu zal er gebruik gemaakt worden van de functie "tf.random.log_uniform_candidate_sampler"
om een aantal (num_ns) negatieve samples voor een gegeven target woord in een window te krijgen.
Voor het trainen kan het.

In [13]:
target_word, context_word = positive_skip_grams[0]

num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,
    name="negative_sampling"
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])


tf.Tensor([7 1 6 0], shape=(4,), dtype=int64)
['het', 'hallo,', 'om', '<pad>']


In [14]:
squeezed_context_class = tf.squeeze(context_class, 1)

context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word

In [15]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 3
target_word     : is
context_indices : [4 7 1 6 0]
context_words   : ['een', 'het', 'hallo,', 'om', '<pad>']
label           : [1 0 0 0 0]


In [16]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [17]:
def generate_training_data(sequences, window_size, num_ns, vocab_size):
  targets, contexts, labels = [], [], []

  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  for sequence in tqdm.tqdm(sequences):

    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=10)

    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          name="negative_sampling")

      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

## Word2Vec:

Het word2vec algorithme is een word embedding techniek in natural language processing die er voor zorgt dat woorden als vectors gerepresenteerd kunnen worden in een continue vector ruimte. 
Dit kan vervolgens gebruikt worden om relaties tussen woorden te achterhalen.



Hier zal de dataset ingeladen worden:
Lees dataset regel voor regel en verwijder lege regels

In [18]:
#text_ds = tf.data.TextLineDataset(["/run/media/floris/FLORIS_3/Data_set/PubMed/dataset_100914.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))
text_ds = tf.data.TextLineDataset(["/home/floris/Documenten/Dataset/trainingsData.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))

# Verwijder punctuatie / hoofdletters.
def standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

Leer de vocabulary van de data, de top 20000 meest frequente woorden krijgen een nummer.

In [19]:
# Aantal woorden in seq en vocabulary grootte:
sequence_length = 200
vocab_size = 20000

# Vectorize the layer en split en map strings tokens met TextVectorization:
vectorize_layer = layers.TextVectorization(
    standardize=standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

Batch de data voor snellere verwerking:

In [20]:
vectorize_layer.adapt(text_ds.batch(BATCH_SIZE))

2026-01-29 20:21:01.789942: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Gebruik de originele traininstext en gebruik de "vectorize_layer" met integers voor alle 20000 meest voorkomende tokens. (in dit geval woorden) Nu is er dus een array met integers die deze woorden representeren.

In [21]:
# Vectoriseer de data in text_ds:
text_vector_ds = text_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [22]:
# Maak een lijst van deze gevectoriseerde data:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

6933


2026-01-29 20:21:02.673017: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


De functie generate_training_data() maakt eerst een skip-gram, het kiest eerst een target woord
en kijkt binnen het gekozen window, bijvoorbeeld 5 woorden.
Als het andere woord binnen de window voorkomt kan een positief paar gevormd worden. 
dit zijn de integers waaruit de vocabulary bestaat, bijvoorbeeld (4,3).

Als het model alleen positieve voorbeelden zou krijgen zou het niet kunnen leren omdat het alles als positief zou classificeren. Om dit tegen te gaan worden ook negatieve paren gegenereerd. Dit zijn woorden die niet binnen het context window samen voorkomen. 
Deze woorden worden willekeurig gekozen.

Later in de functie worden de ware paren en de negatieve paren gecombineerd in een lijst, de context lijst en de even lange "labels" lijst die voor elke waarde in de context lijst een 0 of een 1 bevat, 0 voor een negatief paar en 1 voor een positief paar. 

Alle target woorden komen meerdere keren voor in de targets lijst zodat het model er goed van kan leren.

In [23]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=5,
    num_ns=5,
    vocab_size=vocab_size)

# print(f"targets.shape: {targets.shape}")
# print(f"contexts.shape: {contexts.shape}")
# print(f"labels.shape: {labels.shape}")

100%|██████████| 6933/6933 [37:44<00:00,  3.06it/s]  


Gegenereerde data opslaan:
Het duurde lang om te genereren dus sla ik het op in een .joblib bestand, dit zal ik ook voor sommige vervolg stappen doen.


In [24]:
# joblib_list = [targets, contexts, labels]

# joblib.dump(joblib_list, "targets_contexts_labels.joblib")

Maak de dataset die gebruikt zal worden voor het trainen:
Eerst wordt er een tuple gemaakt van het target woord, de context en de labels.
Voor het trainen zal de X dus de tuple zijn en de y de labels.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

Het Word2Vec model dat gebruikt zal worden voor het trainen bestaat uit twee
embedding layers met als input layer (ter grootte van het aantal samples per batch).


In [ ]:
# class Word2Vec(tf.keras.Model):
#   def __init__(self, vocab_size, embedding_dim):
#     super(Word2Vec, self).__init__()
#     self.target_embedding = layers.Embedding(vocab_size,embedding_dim,name="w2v_embedding")
#     self.context_embedding = layers.Embedding(vocab_size, embedding_dim)

#   def call(self, pair):
#     target, context = pair
#     # context: (batch, context)
#     if len(target.shape) == 2:
#       target = tf.squeeze(target, axis=1)
#     # target: (batch,)
#     word_emb = self.target_embedding(target)
#     # word_emb: (batch, embed)
#     context_emb = self.context_embedding(context)
#     # context_emb: (batch, context, embed)
#     dots = tf.einsum('be,bce->bc', word_emb, context_emb)
#     # dots: (batch, context)
#     return dots

In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,embedding_dim,name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size, embedding_dim)

    def call(self, pair):
        target, context = pair
        if len(target.shape) == 2:
            target = tf.squeeze(target, axis=1)

        word_emb = self.target_embedding(target)
        context_emb = self.context_embedding(context)

        # Normaliseer vectoren naar lengte 1:
        word_emb = tf.nn.l2_normalize(word_emb, axis=-1)
        context_emb = tf.nn.l2_normalize(context_emb, axis=-1)
        dots = tf.einsum('be,bce->bc', word_emb, context_emb)
        return dots

## Verschil tussen maten van gelijkenis tussen vectoren

De bovenstaande functie gebruikt het dot product om gelijkheid tussen de vectoren te berekenen.
Dit is niet de enige methode die gebruikt kan worden. Een andere die veel gebruikt is bij machine learning is cosine similarity, ook wordt de euclidische afstand wel gebruikt. 

Het dot product, dat gebruikt wordt in het bovenstaande codeblok (afkomstig van een tensorflow voorbeeld op https://www.tensorflow.org/) vermenigvuldiged de nummers in de vectors en sommeerd ze.
Het is een maat die de richting van de vector en de magnitude bevat. Er onder staat een test waar ik het dot product vervangen heb door cosine similarity.
Dit is ook sneller om door de computer uit te laten rekenen dat bijvoorbeeld cosine similarity.
Bij embedding modellen hebben woorden die frequent voorkomen een langere vector (grotere magnitude)

Cosine similarity is vergelijkbaar met het dot product maar dan gedeeld door de lengte van de vectors. De lengte van alle vectoren zal dus 1 zijn en alleen de richting van de vectoren is een factor.

Hier wordt het Word2Vec algorithme gebruikt
Hier is de [Adam](https://arxiv.org/abs/1412.6980) optimizer gebruikt, dit is een algorithme dat de learning rate aan kan passen. 

In [ ]:
embedding_dim = 48
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam', 
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
# Om tensorboard te kunnen gebruiken om de trainingsvoortgang te kunnen zien:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

### Train model:

Het model zal nu getrained worden met als loss functie categoricalcrossentropy om de prestaties van het model te scoren. Het model krijgt veel meer negatieve samples dan positieve dus moet het zeker zijn in de keuzes. Het embedden van tekst is dus een classificatie probleem geworden.
De softmax functie veranderd de scores in percentages met een totaal van 100%. (De kans dat het label correct is)


In [ ]:
word2vec.fit(dataset, epochs=50, callbacks=[tensorboard_callback])

Epoch 1/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 361s 7ms/step - accuracy: 0.5895 - loss: 1.0613
Epoch 2/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 350s 7ms/step - accuracy: 0.5973 - loss: 1.0306
Epoch 3/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 349s 7ms/step - accuracy: 0.6085 - loss: 1.0099
Epoch 4/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 348s 7ms/step - accuracy: 0.6189 - loss: 0.9922
Epoch 5/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 347s 7ms/step - accuracy: 0.6255 - loss: 0.9817
Epoch 6/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 349s 7ms/step - accuracy: 0.6299 - loss: 0.9751
Epoch 7/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 349s 7ms/step - accuracy: 0.6331 - loss: 0.9705
Epoch 8/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 348s 7ms/step - accuracy: 0.6355 - loss: 0.9671
Epoch 9/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 349s 7ms/step - accuracy: 0.6374 - loss: 0.9644
Epoch 10/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 348s 7ms/step - accuracy: 0.6388 - loss: 0.9623
Epoch 11/50
49020/49020 ━━━━━━━━━━━━━━━━━━━━ 346s 7ms/step - accuracy: 0.6401 -

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
joblib_list = [weights, vocab]

joblib.dump(joblib_list, "weights_vocab.joblib")

['weights_vocab.joblib']

In [ ]:
num_tokens = min(len(vocab), len(weights))

with io.open('vectors.tsv', 'w', encoding='utf-8') as out_v, \
     io.open('metadata.tsv', 'w', encoding='utf-8') as out_m:

    # Skip token 0
    for index in range(1, num_tokens):
        word = vocab[index]
        vec = weights[index]
        
        clean_word = word.strip()
        
        if not clean_word:
            clean_word = "[UNK_EMPTY]"

        # Vector
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        
        # Metadata
        out_m.write(clean_word + "\n")

print(f"{num_tokens - 1} vectors en metadata entries.")

19999 vectors en metadata entries.


## Trainen van het model:

In [ ]:
# model = EmbeddingModel.EmbeddingModel()
# model.load_dataset()
# model.create_vocab()
# model.vectorize()
# model.generate_trainingdata()
# model.word2vec()
# model.fit()
# model.save2file()

## Data:
Als trainingsdata gebruik ik 869597 Pubmed artikelen (~70GB) die ik gedownload heb van https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/

oa_comm_xml.PMC000xxxxxx.baseline.2025-06-26.tar.gz t/m oa_comm_xml.PMC004xxxxxx.baseline.2025-06-26.tar.gz

Deze zijn nieuwer dan de artikelen uit 2021 die op /commons/data/NCBI/PubMed/ staan.

Vervolgens moeten deze artikelen doorzocht worden op inhoud. \
De .xml bestanden hebben headers voor titel, abstract, body etc. deze doorzoek ik met behulp van mijn klasse: data_parser.py

Het filteren op de drie keywords "cancer", "melanoma" en "carcinoma" duurde 9 uur en 30 minuten en reduceerde de hoeveelheid artikelen 
van 869597 naar 100914 (2.7GB).

100914 Artikelen met deze zoekcriteria opgeslagen in: /run/media/floris/FLORIS_3/Data_set/PubMed/dataset_large.txt

Voor lateren trainingsruns heb ik de data_parser.py aangepast zodat hij inplaats van het hele artikel op een enkele regel zet, een alinea per regel zet.


In [ ]:
keyword_list = ["cancer", "melanoma", "carcinoma"] # Niet hoofdlettergevoelig...

trainings_data = xml_parser("/home/floris/Documenten/Dataset/PMC000xxxxxx", "/home/floris/Documenten/Dataset/trainingsData.txt", keyword_list)

# Filter de data:
trainings_data.run()



3028 XML bestanden...


100%|██████████| 3028/3028 [00:50<00:00, 59.61it/s]

Voltooid...
194 Artikelen met deze zoekcriteria opgeslagen in: /home/floris/Documenten/Dataset/trainingsData.txt


Hierboven staat de uitkomst van de filterstap, van de kleinere subset van de trainingsdata zijn 194 artikelen gevonden die een (of meer) van de drie keywords bevatten in het abstract. 
Dit kwam uit op een .txt bestand van ~4.1MB maar door het genereren van de vele positieve en negatieve skip-gram combinaties vulde dit tijdens het trainen vaak het RAM geheugen van mijn laptop volledig. (64GB)
 

## Resultaten:

Het eerste model was met de volgende hyperparameters getrained:

vocabulary grootte: 20000 \
context lengte: 100 \
window_size=5 \
num_ns=4 \
dims = 128

Het model was erg overfit op de trainingsdata, de loss was bijna 0 en accuratesse bijna 1. \
De volgende poging zal een minder grote vocabulary krijgen en minder dimenties om beter te generaliseren i.p.v. de trainingsdata te onthouden. 

De onderstaande afbeelding weergeeft de loss en accuracy grafieken met Tensorboard:

![](img/run_1.png)


Zoals te verwachten is wordt de loss steeds lager.

### Tweede run:

Na het proberen van vele combinaties van hyperparameters lijken deze een veel beter resultaat te geven vergeleken met de vorige:

vocabulary grootte: 1000 \
context lengte: 1000 \
window_size=5 \
num_ns=4 \
dims = 64

![](img/run_2.png)


Training accuratesse lijkt te convergeren rondt 0.62, voor meer epochs trainen zou weinig zin hebben. Het zelfde lijkt voor de loss te gebeuren.

### Derde run:


vocabulary grootte: 20000 \
context lengte: 300 \
window_size=5 \
num_ns=5 \
dims = 48


![run_3.png](img/run_3.png)


De derde traininsronde presteerd het beste na de resulterende vectors.tsv en metadata.tsv in de projector.tensorflow.org website te visualiseren en de gelijkenis tussen een aantal woorden op te zoeken. 
De voorgaande modelen lijken veel meer willekeurig lijkende woorden in de lijst te hebben dan dit model. 

Wanneer we naar de nabijgelegen woorden in de vectorspace kijken zijn de woorden die er het dichtst bij liggen "ductal", "frequency" en "invasion". 
Een "ductal carcinoma" is een vroege form van borstkanker en aangezien het woord ductal waarschijnlijk niet al te vaak voorkomt in andere context (in deze artikelen) zal het vrijwel altijd met kanker geassocieerd worden.
Het woord ductal kwam 36 keer voor in de trainingsdata.

Het woord "malignency" heeft het woord "cancers" maar ook "tyrosine" , misschien komt dit woord er voor omdat tyrosine kinase inhibitors ook tegen kanker gebruikt kunnen worden.

Deze laatste trainingsronde heb ik met een kleinere subset van de artikelen gemaakt dan de eerste. Het gefilterde .txt bestand bevatte maar 197 artikelen.

Het aantal regels: `wc -l trainingData.txt` = 6934 regels \
Aantal woorden `wc -w trainingData.txt` = 607965 woorden \

Voor de keywords waarop ik gefilterd heb, "cancer", "melanoma" en "carcinoma", dit is hoeveel zinnen deze woorden bevatten: \
`grep -o "cancer" trainingsData.txt | wc -l` = 4511 \
`grep -o "melanoma" trainingsData.txt | wc -l` = 588 \
`grep -o "carcinoma" trainingsData.txt | wc -l` = 1137 \
 

### Frequentie analyse trainingstekst:

`cat trainingsData.txt | tr -s ' ' '\n' | sort | uniq -c | sort -nr | head -50` =

```
  27802 the
  24475 of
  17275 and
  14315 in
  11189 to
   8467 a
   6657 with
   6243 for
   5475 were
   5346 was
   4782 that
   4563 is
   4321 The
   3910 by
   3545 as
   3086 cancer
   2950 or
   2854 be
   2826 from
   2671 are
   2635 cells
   2347 on
   2341 at
   2235 patients
   2149 not
   2129 cell
   2056 tumor
   1880 this
   1826 have
   1703 In
   1675 an
   1482 expression
   1365 which
   1322 been
   1258 these
   1203 between
   1192 we
   1189 may
   1175 has
   1112 using
   1111 than
   1105 study
   1103 also
   1094 data
    986 breast
    979 used
    918 This
    915 more
    910 all
    901 can

```

Dit bovenstaande bash statement wordt elk woord in een nieuwe regel gezet, het blijkt dus dat de "stopwoorden" die in normale zinnen voorkomen. Dit is voor de 50 meest frequent voorkomende woorden gedaan. 
Ik heb uit verschillende bronnen tegensprekende informatie gevonden ([Verwijderen van stopwoorden]("https://proceedings.neurips.cc/paper_files/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf") en [Niet verwijderen van stopwoorden]("https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html")) over het nut van het verwijderen van stopwoorden. \
Uiteindelijk heb ik er voor gekozen om ze niet te verwijderen omdat deze woorden toch waarschijnlijk een belangrijke rol spelen in de relaties tussen woorden in zinnen. \
Ook het keyword "cancer" waar ik op gefilderd heb komt voor in de top 50, mijn speculatie is dat het nu zo onevenredig vaak voorkomt dat het woord geassocieerd wordt met te veel andere woorden. 



# Conclusie / Reflectie:

Het Vec2Word embedding model dat resulteerde van de trainingscode in dit notebook presteerd minder goed dan gehoopt. Het derde model was het beste met de laagste loss en hoogste acuratesse. Dit model was getrained voor 50 epochs tegenover de 20 van de eerste. De tweede had ook voor 50 epochs getrained maar de slechte prestaties kunnen mogelijk toegewezen worden aan de te kleine vocabulary, het model kent niet genoeg woorden om toevallig de woorden er bij te hebben die vaak genoeg voorkwamen toe te voegen aan de vocab. (in de top 1000 van meeste frequente woorden omdat de vocabulary grootte 1000 was.)
Na het bekijken van de woorden die het meeste overeenkomst vertonen met elkaar, volgens het model zaten veel termen die wel verwant zijn aan welkaar
maar er zat ook veel, op het oog onzinnige data tussen die niets te maken heeft met het gekozen target woord. 

Er zijn nog vele andere dingen die gedaan kunnen worden om potentieel het model beter te laten werken zoals: beter filteren, bijvoorbeeld op meer termen in plaats van de drie die nu gebruikt worden. 
Omdat het trainen lang duurde is het niet goed mogelijk om een soort grid search voor de optimale hyperparameters uit te voeren maar er zijn wel enkele combinaties die ik, als er meer tijd was, zou willen proberen zoals de context window lengte korter maken zodat het model minder goed verbanden tussen complexere concepten zou "begrijpen" maar dat woorden die bijna altijd samen voorkomen ook beter terug komen. Zo zou een weinig voorkomend woord zoals "ductal" in deze artikelen bijna altijd voorkomen in de combinatie ductal carcinoma terwijl dit nu niet zo is bij vergelijkbare woorden. Het woord ductal zelf kwam niet vaak genoeg voor om een eigen nummer te krijgen en zou dus een "UNK" (unknown) token krijgen. 

Een ander punt is dat ik een andere tokenzer had kunnen gebruiken zoals byte pair encoding waar substring die vaken voorkomen in verschillende woorden gebruikt worden als tokens zodat de vocabulary minder groot hoeft te zijn om de zelfde woorden / texten te bevatten.
Verder speculeer ik dat het belangrijkste punt dat het model beter zou kunnen maken het gebruiken van meer trainingsdata is. Ik had in totaal ongeveer 70GB aan Pubmed artikelen gedownload maar hier heb ik uiteidelijk maar een kleine subset van gebruikt omdat de trainingstijd enorm lang is. Maar door geen gebruik te maken van batches en niet de volledige trainingsdataset in RAM geheugen te laden zou het wel mogelijk zijn om het model te trainen met de volledige dataset. Nvidia CUDA zou het trainen ook sneller moeten maken maar dat werkte niet altijd in dit notebook. Ook was het verschil in trainingssnelheid minder groot dan verwacht op een Nvidia RTX 3070.

Nog een ander onderdeel waar ik meer mee had kunnen experimenteren was het gebruiken van een andere optimizer dan Adam, een andere loss functie dan CrossEntropyloss en andere vector gelijkenis functies dan het dot product.